<a href="https://www.kaggle.com/code/juwaynkeane/bank-churn?scriptVersionId=144559754" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_csv("/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv")
df.sample(5)

In [ ]:
df.dtypes

In [ ]:
#GET RID OF USELESS COLUmNS
df.drop('Surname',axis='columns',inplace=True)
df.drop('CustomerId',axis='columns',inplace=True)

In [ ]:
df.IsActiveMember.values
df[df.IsActiveMember==0]

In [ ]:
df.shape

In [ ]:
#deletes all the rows with space
df[df.IsActiveMember!=0].shape


In [ ]:
#loyal
tenure_churn_no = df[df.Exited==0].Tenure
#leavers
tenure_churn_yes = df[df.Exited==1].Tenure
#some data visualisation for you guys
plt.xlabel("tenure")
plt.ylabel("Number Of Active Customers")
plt.title("Customer Churn Prediction Visualiztion")


plt.hist([tenure_churn_yes, tenure_churn_no], rwidth=0.95, color=['green','red'],label=['Churn=Yes','Churn=No'])
plt.legend()

In [ ]:
#loyal
tenure_churn_no = df[df.Exited==0].EstimatedSalary
#leavers
tenure_churn_yes = df[df.Exited==1].EstimatedSalary
#some data visualisation for you guys
plt.xlabel("EstimatedSalary")
plt.ylabel("Number Of Active Customers")
plt.title("Customer Churn Prediction Visualiztion")


plt.hist([tenure_churn_yes, tenure_churn_no], rwidth=0.95, color=['green','red'],label=['Churn=Yes','Churn=No'])
plt.legend()

In [ ]:
#tells me unique columns
for column in df:
    print(f'{column}:{df[column].unique()}')

In [ ]:
def print_unique_col_values(df):
    #for loop in every column
       for column in df:
            if df[column].dtypes=='object':
                print(f'{column}: {df[column].unique()}') 

In [ ]:
print_unique_col_values(df)

In [ ]:
df['Gender'].replace({'Female':1,'Male':0},inplace=True)

In [ ]:
#Ml mdels need yes and no to be 1&0
df1 = pd.get_dummies(data=df, columns=['Geography'])
df1.columns

In [ ]:
df1.sample(5)

In [ ]:
df1.dtypes

In [ ]:
#since not 0 and 1 we need to scale them
cols_to_scale = ['Tenure','Balance','Age','EstimatedSalary','CreditScore','NumOfProducts']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1[cols_to_scale] = scaler.fit_transform(df1[cols_to_scale])

In [ ]:
for col in df1:
    print(f'{col}: {df1[col].unique()}')

In [ ]:
#pick x and y
X = df1.drop('Exited',axis='columns')
y = df1['Exited']
#splits dataset
from sklearn.model_selection import train_test_split
#80 train 20 test `
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

In [ ]:
X_test.shape

In [ ]:
len(X_train.columns)

In [ ]:
import tensorflow as tf
from tensorflow import keras


model = keras.Sequential([
    #same number of nuerons as columns, for hidden layers relu is easy to compute, but can have 20 nuerons if you want
    keras.layers.Dense(10, input_shape=(13,), activation='relu'),
    #just wnat less than input, knows what the input shape, can remove this 
    keras.layers.Dense(8, activation='relu'),
    #1 and o so sigmoid 
    keras.layers.Dense(1, activation='sigmoid')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
#layers can be played with to increase accuracy
#start with 5 epoch to get an idea
model.fit(X_train, y_train, epochs=100)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
#y predictor
yp = model.predict(X_test)
yp[:5]

In [ ]:
#want yp to be 1 dimensional and be 0 and 1 like sigmoid
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))
#precision 0 people who left

In [ ]:
import seaborn as sn
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_pred)

plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')